# Предсказание коэффициента восстановления золота из золотосодержащей руды

## Описание проекта.

Необходимо подготовить прототип модели машинного обучения для «Цифры». Компания разрабатывает решения для эффективной работы промышленных предприятий.

Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. В нашем распоряжении данные с параметрами добычи и очистки.

Модель поможет оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками.
Нам нужно:
* Подготовить данные
* Провести исследовательский анализ данных
* Построить и обучить модель

Целевыми признаками для нас будут эффективность обогащения чернового концентрата - rougher.output.recovery
и эффективность обогащения финального кнцентрата - final.output.recovery


***Технологический процесс получения золота.***

<img src="https://pictures.s3.yandex.net/resources/viruchka_1576238830.jpg" title="Технологический процесс получения золота."/>

Смесь золотоносной руды подаётся во флотационную установку. После обогащения получается черновой концентрат и «отвальные хвосты», то есть остатки продукта с низкой концентрацией ценных металлов. На стабильность этого процесса влияет непостоянное и неоптимальное физико-химическое состояние флотационной пульпы (смеси твёрдых частиц и жидкости). Черновой концентрат проходит два этапа очистки. На выходе получается финальный концентрат и новые отвальные хвосты.

## Шаг 1. Откроем файлы с данными, изучим общую информацию, сделаем предобработку данных.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
import warnings
warnings.filterwarnings('ignore')

Посмотрим на обучающую выборку.

In [ ]:
train_sample = pd.read_csv('gold_recovery_train_new.csv')
train_sample.head()

In [ ]:
train_sample.info()

Обучающая выборка обладает 87 признаками - 1 - это дата по часам, 86 - производственные характеристики.

Такое количество обусловлено тем, что технология производства золота делится на 4 этапа:

* rougher - флотация;
* primary_cleaner — первичная очистка
* secondary_cleaner — вторичная очистка
* final — характеристики конечного продукта.

Каждый этап производства золота, в свою очередь, обладает своими типами параметров, их тоже четыре:

* input — параметры сырья;
* output — параметры продукта;
* state — параметры, характеризующие текущее состояние этапа;
* calculation — расчётные характеристики.

Каждый тип параметров включает в себя уже конкретные параметры, которые выражают определенные производственные характеристики, их много, поэтому весь список приводить не будем.

Пример: rougher.input.feed_au - доля золота в руде до осуществления флотации, а rougher.output.concentrate_au - доля золота в концентрате после флотации.

Также обращает на себя внимание большое количество пропусков в данных, и во многих столбцах количество пропущенных значений различается. Кроме того, самое большое количество пропусков в выходных значениях содержания золота в хвостах после флотации (rougher.output.tail_au) и в эффективности обогащения чернового концентрата - rougher.output.recovery.

Далее посмотрим на тестовую выборку.

In [ ]:
test_sample = pd.read_csv('gold_recovery_test_new.csv')
test_sample.head()

In [ ]:
test_sample.info()

Здесь мы видим меньшее количество признаков и гораздо меньшее количество пропусков.

Далее посмотрим на исходные данные.

In [ ]:
full_data = pd.read_csv('gold_recovery_full_new.csv')
full_data.head()

In [ ]:
full_data.info()

Признаков также, как и в тестовой выборке 87. 
Проверим, что названия столбцов в обучающей выборке соответствуют исходным данным.

In [ ]:
full_data.columns == train_sample.columns

Итак, мы здесь видим полное соответствие названий.

Сумма значений rougher.input.feed_au в обучающей выборке (14149) и в тестовой (5290) равно количеству значений в исходных данных (19439). То есть, по всей видимости, заполнить пропуски значениями из исходных данных в обучающей выборке у нас не получится.

### Проверим эффективности обогащения.

Проверим, верно ли рассчитана эффективность обогащения чернового концентрата rougher.output.recovery.

Для этого возьмем обучающую выборку, самостоятельно рассчитаем этот коэффициент по формуле и найдем среднее абсолютное отклонение (МАЕ) между расчетами и фактическими значениями.

Создадим датафрейм для проверки только с необходимыми признаками.

In [ ]:
recovery_check_data = train_sample[['date', 'rougher.output.concentrate_au', 'rougher.input.feed_au',
                                   'rougher.output.tail_au', 'rougher.output.recovery' ]]
recovery_check_data.info()

Создадим функцию для расчета эффективности.
Эффективность обогащения рассчитывается по формуле:

<img src="https://pictures.s3.yandex.net/resources/Recovery_1576238822.jpg" title="Эффективность обогащения"/>

где:
* C — доля золота в концентрате после флотации/очистки;
* F — доля золота в сырье/концентрате до флотации/очистки;
* T — доля золота в отвальных хвостах после флотации/очистки.

In [ ]:
def check_rougher_output_recovery(row):
    
    C = row['rougher.output.concentrate_au']
    F = row['rougher.input.feed_au']
    T = row['rougher.output.tail_au']
    
    if F * (C - T) == 0:
        return
    else: 
        recovery = (C * (F - T)) / (F * (C - T)) * 100
        
        return recovery

In [ ]:
recovery_check_data['check_rougher_recovery'] = recovery_check_data.apply(check_rougher_output_recovery, axis = 1)
mean_absolute_error(recovery_check_data['rougher.output.recovery'],recovery_check_data['check_rougher_recovery'] )

Итак, МАЕ находится на очень низком уровне, что говорит о том, что фактические данные об эффективности обогащения чернового концентрата в обучающей выборке рассчитаны верно. Значит, мы можем доверять фактическом коэффициенту.

### Анализ признаков тестовой выборки.

Выясним, какие из признаков, которые есть в исходных данных, отсутствуют в тестовой выборке. Для ответа на этот вопрос сравним множества признаков двух датасетов.

In [ ]:
set(train_sample.columns) ^ set(test_sample.columns)

Здесь мы видим, что в тестовой выборке отсутствуют признаки со следующими типами параметров:

* calculation - то есть расчетные характеристики;
* output - параметры выходного продукта на каждой стадии.

Другими словами, тестовая выборка содержит только типы параметров:

* input - параметры сырья;
* state - параметры, характеризующие текущее состояние этапа.

### Предобработка данных.

Начнем с индексации. При описани данных нам дали подсказку о том, что данные индексируются датой и временем получения информации. Возможно, мы можем сделать столбец 'date' индексом. Убедимся, что в каждом датасете значения в этом столбце уникальны.

In [ ]:
dfs = [train_sample, test_sample, full_data]
for df in dfs:
    display(df['date'].value_counts())

Видим, что все значения даты уникальны и мы можем сделать этот столбец индексом.

In [ ]:
train_sample = train_sample.set_index('date')
test_sample = test_sample.set_index('date')
full_data = full_data.set_index('date')

Проверим, не содержат ли наши данные дубликатов.

In [ ]:
for df in dfs:
    display(df[df.duplicated()])

Дубликатов не выявлено.

Теперь самая главная проблема - большое количество пропусков, принимая во внимание тот факт, что у нас есть два целевых признака - rougher.output.recovery и final.output.recovery, выглядит опасным заполнять какими-либо данными признаки, которые непосредственно влияют на целевые признаки, так как, на наш взгляд, это будет похоже на то, что мы немного "сочиним" ответы, что приведет к некорректному анализу.

Поэтому наблюдения с пропусками в признаках, которые непосредственно влияют на rougher.output.recovery (группа признаков rougher.output) и на final.output.recovery (группа признаков final.output) решено удалить.

Найдем названия этих столбцов и сохраним в массив.

In [ ]:
output_arrays = (list(full_data.filter(regex=("final.output.")).columns) +
                list(full_data.filter(regex=("rougher.output.")).columns))
output_arrays

Итак, у нас есть массив с названия столбцов, которые выражают характеристики выхода продукции - на этапе флотации и после вторичной очистки.

Значит, из наших датасетов будем удалять строки с пропусками в этих столбцах.

ЧТо касается остальных признаков, то в описании данных также содержится подсказка о том, что соседние по времени данные часто похожи. Попробуем опереться на нее и заполнить такие пропуски последним наблюдаемым значением. Для этого выберем метод ffill в функции fillna().

Сделаем это для train_sample и full_data.

In [ ]:
train_sample = train_sample.dropna(subset=output_arrays).fillna(method='ffill')
train_sample.info()

In [ ]:
full_data = full_data.dropna(subset=output_arrays).fillna(method='ffill')
full_data.info()

Для тестовой выборки просто заполним пропуски методом ffill, поскольку она не имеет характеристик выхода продукта.

In [ ]:
test_sample = test_sample.fillna(method='ffill')
test_sample.info()

В конце взглянем на итоговую описательную статистику.

In [ ]:
full_data[['rougher.input.feed_au','rougher.input.feed_size','rougher.output.concentrate_au',
             'rougher.output.tail_au', 'rougher.output.recovery',
              'primary_cleaner.output.concentrate_au','primary_cleaner.output.tail_au', 
              'secondary_cleaner.output.tail_au', 'final.output.concentrate_au',
         'final.output.tail_au', 'final.output.recovery']].describe().T

В таблице настораживают 2 момента - в столбцах rougher.output.recovery и final.output.recovery есть значения с 0 и 100 - то есть когда металл был восстановлен из сырья полностью и вообще не восстановлен.

Нулевая эффективность может быть, когда концентрат равен нулю, а 100% - когда нулю равны соответствующие хвосты.

Проверим, выполняются ли эти условия для обозначенных случаев.

In [ ]:
full_data[full_data['rougher.output.recovery'] == 0][['rougher.output.concentrate_au']].sum()

Сумма значений столбца равна нулю, значит, в черновом концентрате золота не было, и мы оставим эти строки.

Проверим 100% эффективность.

In [ ]:
full_data[full_data['rougher.output.recovery'] == 100][['rougher.input.feed_au',
                                                        'rougher.output.concentrate_au', 
                                                        'rougher.output.tail_au']]

Такой ситуации не может быть, ведь если есть металл в "хвосте", значит, он отделился от сырья, и в черновом концентрате его доля должна была вырасти. Нужно удалить эти строки.



In [ ]:
full_data = full_data[full_data['rougher.output.recovery'] < 100]
train_sample = train_sample[train_sample['rougher.output.recovery'] < 100]

Проверим нулевую концентрацию для эффективности финального концентрата.

In [ ]:
full_data[full_data['final.output.recovery'] == 0][['final.output.concentrate_au']].sum()

Cумма хвоствых значений равна нулю. Такая ситуация теоретически возможна. Строки оставим.

Поскольку мы удалили строки из исходных данных, удостоверимся в том, что тестовая выборка соответствует индексам исходных данных

In [ ]:
test_sample = test_sample.loc[test_sample.index & full_data.index]
test_sample

Выводы:

* Познакомились с данными, выяснили, что у нас 86 признаков, характеризующих процесс производства золота.

* В тестовой выборке отсутствуют признаки, относящиеся к типу параметров состояния и параметров выходного продукта.

* Проверили расчет эффективности обогащения руды флотацией, выяснили, что данным можно доверять.

* Проиндексировали таблицы столбцом с датой, удалили наблюдения из обучающей выборки и исходных данных там, где есть пропуски в столбцах, непосредственно влияющих на наши целевые признаки, заполнили оставшиеся пропуски последним наблюдаемым значением.

## Шаг 2. Исследовательский анализ данных.

### Изменение концентрации металлов на этапах очистки.

Для анализа изменения концентрации металлов на разных этапах создадим функцию.

In [ ]:
def concentration(metal):
    
    """Функция на вход принимает строковое название металла, затем отбирает признаки, в которых 
    оно содержится. После этого по заранее заданным индексам строятся массивы признаков для отбора 
    значений из исходных данных. Затем строятся боксплоты и выводится описательная статистика"""
    
    all_features = list(full_data.filter(regex=(metal + '$')).columns)
    
    idx = [4,5,2,0]
    
    feed_and_concentrate_features = [all_features[i] for i in idx]
    
    idx_2 = [6,3,7,1]
    
    tail_features = [all_features[i] for i in idx_2]
    
    sns.set_style('whitegrid')
    ax = plt.subplots(figsize = (10,6))
    data_concentrate = full_data[feed_and_concentrate_features]
    chart = sns.boxplot(data=data_concentrate, orient='h', palette='Set2', fliersize=0.5)
    plt.title(metal)
    plt.xlabel('Stage concentration, %')
    
    ax = plt.subplots(figsize = (10,6))
    data_tail = full_data[tail_features]
    chart_2 = sns.boxplot(data=data_tail, orient='h', palette='Set2', fliersize=0.5).set_xlim([0,25]) 
    plt.title('tail ' + metal)
    plt.xlabel('Stage concentration, %')
    
    display(full_data[feed_and_concentrate_features].describe().T)
    display(full_data[tail_features].describe().T)

Начнем анализ с золота.

In [ ]:
concentration('au')

Доля золота в концентрате на этапе подачи смеси золотосодержащей руды составляет в среднем около 8% и на этапе получения финального концентрата составляет в среднем около 44%. При этом видно, что его доля возрастает от этапа к этапу достаточно равномерно.

Стоит обратить внимание, что концентрации имеют очень много выбросов "вниз", то есть много значений выходят за нижний придел 1.5 межквартильного интервала.

Что касается доли золота в отвальных хвостах - то она растет до этапа вторичной очистки. Но не превышает 5%. Распределения характеризуются многочисленными выбросами вверх - значит, имеют место быть ситуации выпадения в хвосты чрезмерного количества золота.

Посмотрим на серебро.

In [ ]:
concentration('ag')

На уровне подаваемого сырья концетрации золота и серебра примерно одинаковы - 8.3% против 8.8%. Но, видимо, из-за того, что золото ценнее серебра, мы сосредотачиваемся на получении золота - и доля серебра в черновом концентрате увеличивается, по сравнению с сырьем, но потом на стадии первичной очистки значительное количество серебра уходит в "хвост" - его доля в "хвостах" превышает 15% - более чем на 10 п.п. больше, чем концентрация золота в "хвостах". В итоге в финальном концентрате доля серебра составляет в среднем 5%.

Рассмотрим свинец.

In [ ]:
concentration('pb')

Исходя из графиков и таблиц заметно, что свинец, по всей видимости, является трудноудалимой примесью, с которой борются уже на стадиях после получения финального концентрата. Доказательством этому служит его возрастающая доля по этапам: в начальном сырье его всего лишь 3.5%, но в финальном концентрате - целых 10%.
Отметим, что, к примеру, после флотации его доля в хвостах около 0.5%.

Итого, мы выяснили, что доля золота во время этапов возрасате до 44%, по сравнению с сырьем, где оно составляет 8%. Доля серебра в финальном концентрате мала, вероятно, это обусловлено свойствами руды. Также мы выяснили, что свинец является трудноудалимой примесью.

### Сравнение распределений размеров гранул сырья.

Cравним, различается ли распределение гранул сырья на обучающей и тестовой выборках.

Построим гистограммы.

In [ ]:
fig, axes = plt.subplots(figsize=(12,7))

chart = sns.distplot(train_sample['rougher.input.feed_size'],ax=axes, color='y',
                     bins=100, label='Обучающая выборка').set_xlim([20,120])

chart_2 = sns.distplot(test_sample['rougher.input.feed_size'],ax=axes, color='b',
                       bins=100, label='Тестовая выборка').set_xlim([20,120])

plt.title('Распределение размеров гранул сырья на выборках')
plt.xlabel('Размер гранул сырья')
axes.legend();

In [ ]:
display(train_sample[['rougher.input.feed_size']].describe().T)
display(test_sample[['rougher.input.feed_size']].describe().T)

Распределение размеров все-таки незначительно отличается между выборками. Видно, что на тестовой выборке больше значений размера в диапазоне 40-50, чем на тестовой. На тестовой, в свою очередь - больше значений в диапзоне 50-60. То есть у распределений немного не совпадают пики. Поэтому медиана обучающей выборки на 5 больше, чем тестовой.

### Исследование суммарной концентрации всех веществ на разных этапах.

Исследуем суммарную концентрацию веществ на этапе подачи сырья, чернового концентрата и финального концентрата.
Для этого мы будем суммировать 4 столбца, оканчивающиеся на соответствующих этапах на '_au', '_ag', '_pb', '_sol'.

In [ ]:
def overall_feature(stage, df):
    
    """Функция принимает на вход строковое название интересующего нас этапа и нужную выборку.
    Отбираются признаки для суммирования, суммируются и записываются новым столбцом в нужную выборку.
    Затем по этому столбцу строится диаграмма и рассчитывается количество значений равных 0 и > 100"""
    
    dict_names = {'rougher.input.feed' : 'Этап подачи сырья', 
                  'rougher.output.concentrate': 'Черновой концентрат',
                'final.output.concentrate' : 'Финальный концентрат'}
    
    features = (list(df.filter(regex=(stage+'_..$')).columns) + 
                        list(df.filter(regex=(stage+'_...$')).columns))
    
    elements = df[features]
    df[stage+'.overall_concentration'] = elements.sum(axis=1)
    
    df[stage+'.overall_concentration'].plot(kind='hist', bins=50, range=[0,100], figsize = [9,5])
    
    plt.title(dict_names[stage])
    plt.xlabel('Суммарная концентрация веществ, %')
    plt.ylabel('Частота')
    
    print('Количество наблюдений с концентрацией равной 0%: ',
          len(df[df[stage+'.overall_concentration'] == 0]))
    
    print('Количество наблюдений с концентрацией больше 100%: ',
          len(df[df[stage+'.overall_concentration'] >= 100]))
    
    display(df[[stage+'.overall_concentration']].describe().T)

Начнем с обучающей выборки и этап подачи сырья.

In [ ]:
overall_feature('rougher.input.feed', train_sample)

Ситуация выглядит нормальной - большинство значений расположились в диапазоне от 50 до 70%. Нулевых и зашкаливающих концентраций нет. Значит, предположим, что здесь аномалий нет.

Рассмотрим характеристики чернового концентрата обучающей выборки.

In [ ]:
overall_feature('rougher.output.concentrate', train_sample)

Здесь большинство наблюдений кучно расположились в диапзоне от 55% до 80%. Но присутствуют нулевые значения.
Такие наблюдения можно признать аномальными - поскольку на предыдущем этапе мы явно видели, что вся поступившая руда имела коцентрацию четырех основных веществ.

Можно предположить, что все ушло в "хвосты", однако вряд ли технологический процесс допустил бы такую ситуацию, к тому же здесь значения строго равны нулю - то есть там вообще нет металлов. А самое главное отсутствует параметр '_sol', который, видимо, является заполнителем прочих веществ. Следовательно, это аномальная ситуация и от этих наблюдений нам лучше избавиться.

Рассмотрим характеристики финального концентрата обучающей выборки.

In [ ]:
overall_feature('final.output.concentrate', train_sample)

Здесь видно, что подавляющее большинство наблюдений лежит в диапзоне 60-80%. Также встречается несколько нулей - по причинам описанным выше, эти строки также можно считать аномальными, поэтому удалим их.

In [ ]:
train_sample = train_sample[(train_sample['rougher.output.concentrate.overall_concentration'] > 0) & 
                        (train_sample['final.output.concentrate.overall_concentration'] > 0)]

Далее рассмотрим на предмет концентрации тестовую выборку.
У нее нет признаков с выходными параметрами, поэтому будем рассматривать только параметры сырья.

In [ ]:
overall_feature('rougher.input.feed', test_sample)

Мы видим среднюю концентрацию на уровне обучающей выборки, значений, похожих на аномальные, на наш взгляд, нет. Оставим все строки и удалим столбцы с расчетной концентрацией.

In [ ]:
train_sample = train_sample.drop(columns = list(train_sample.filter(regex=('overall_concentration'))))
test_sample = test_sample.drop(columns = list(test_sample.filter(regex=('overall_concentration'))))

## Шаг 3. Построение и обучение модели.

### Напишем функцию для вычисления итоговой sMAPE.


Перед тем, как написать функцию для оценки качества, подготовим данные для обучения, чтобы понимать, с какими массивами нам нужно будет работать.

По условиям задачи нам необходимо спрогнозировать сразу две величины - эффективность обогащения чернового концентрата ('rougher.output.recovery') и эффективность обогащения финального концентрата('final.output.recovery').

Значит, нам нужна регрессионная модель, в которой будет сразу 2 зависимых переменных.

Создадим массивы для обучающей выборки. Для обучения будем использовать только те признаки, которые есть в тестовой выборке.

In [ ]:
target_train = train_sample[['rougher.output.recovery', 'final.output.recovery']]
features_train = train_sample.loc[:, list(test_sample.columns)]
                               
features_train.info()

Подготовим значения целевых признаков тестовой выборки, отберем их из исходных данных.

In [ ]:
target_test = full_data.loc[test_sample.index, ['rougher.output.recovery', 'final.output.recovery']]
target_test

Для решения задачи прогнозирования эффективности обогащения будем использовать метрику качества - симметричное среднее абсолютное процентное отклонение.

Регрессионные модели в предсказаниях будут возвращать массив, состоящий из подмассивов с двумя элементами, соответственно, первый элемент такого подмассива - это предсказание 'rougher', второй - 'final'.

Учтем это в метрике качества.

In [ ]:
def sMAPE(target, predictions):
    
    """Функция суммирует отношение модуля разности таргета и предсказания к среднему значению
    модуля таргета и модуля предсказания. Затем взвешивает полученные оценки с весами 0.25 и 0.75
    для чернового и финального концентрата соответственно"""
    
    rougher_sMAPE = sum([abs(target.iloc[:,0][i] - predictions[:,0][i]) / 
                         ((abs(target.iloc[:,0][i]) + abs(predictions[:,0][i])) / 2) 
                 for i in range(len(target))]) / len(target) * 100
    
    
    final_sMAPE = sum([abs(target.iloc[:,1][i] - predictions[:,1][i]) / 
                         ((abs(target.iloc[:,1][i]) + abs(predictions[:,1][i])) / 2) 
                 for i in range(len(target))]) / len(target) * 100
    
    overall_sMAPE = 0.25 * rougher_sMAPE + 0.75 * final_sMAPE
    
    return overall_sMAPE

### Обучим модели.

Найдём лучшие модели в алгоритмах линейной регрессии, решающего дерева и случайного леса для регрессии.

Для подбора гиперпараметров будем использовать инструмент GridSearchCV - он позволит перебирать параметры и содержит в себе кросс-валидацию.

Также будем использовать функцию cross_val_score для модели с выбранными параметрами.

Для оценки качества перебираемых параметров будем использовать нашу функцию sMAPE.

Найдя лучшие параметры, применим модель на данных тестовой выборки.

Создадим инструмент оценки качества для его использования в GridSearchCV.

Сделаем это инструментом make_scorer, наша задача минимизировать значение этой метрики, передадим соответствующий параметр.

In [ ]:
my_scorer = make_scorer(sMAPE, greater_is_better=False)

**Линейная регрессия**

In [ ]:
lin_model = LinearRegression()

Проверим качество кросс-валидацией.

In [ ]:
lin_scores = cross_val_score(lin_model, features_train, target_train, scoring=my_scorer, cv=5)
lin_scores

Получили массив sMAPE на 5 валидационных выборках, посчитаем среднее отклонение.

Знак "минус" перед оценками значит, что нам необходимо минимизировать метрику.

In [ ]:
-sum(lin_scores) / len(lin_scores)

В среднем модель линейной регрессии ошибается примерно на 9.5% по взвешенным таргетам.

**Дерево решений**

In [ ]:
tree_model = DecisionTreeRegressor(random_state = 2)

Используем GridSearchCV, чтобы и перебрать немного параметров и получить оценку по кросс-валидации.

In [ ]:
tree_params = {
                'max_features' : range(8, 10, 1),
                'max_depth': range(3,6)}

best_tree = GridSearchCV(tree_model, tree_params, scoring=my_scorer, cv=5,n_jobs=-1, verbose=True)

In [ ]:
best_tree.fit(features_train, target_train)

In [ ]:
best_tree.best_params_

Лучшее дерево обладает глубиной - 3 и число признаков, по которым ищется разбиение равно 9.

In [ ]:
best_tree.best_score_

Оценка лучшей модели чуть выше 8%, что лучше, чем линейная регрессия.

#### Случайный лес

In [ ]:
rf_model = RandomForestRegressor(random_state=2, n_estimators=20)

In [ ]:
forest_params = {
                'max_features' : range(5, 10, 1),
                'max_depth': range(3,5)}

forest_grid = GridSearchCV(rf_model, forest_params, scoring=my_scorer, cv=5, n_jobs=-1, verbose=True)

In [ ]:
forest_grid.fit(features_train, target_train)

In [ ]:
forest_grid.best_params_

In [ ]:
forest_grid.best_score_

У случайного леса глубина - 4, а количество признаков, по которым ищется разбиение - 7. Оценка незначительно лучше, чем у решащего дерева. Получается, что лучшая модель - это случайный лес.

Далее проверим модель на адекватность, то есть узнаем, действительно ли модель справляется с предсказанием значений лучше, чем какая-либо константа, например - медиана целевых признаков.

Для этого создадим массив с медианами.

In [ ]:
target_median_rougher = pd.Series(target_train.iloc[:, 0].median(), index=target_test.index)
target_median_rougher

In [ ]:
target_median_final = pd.Series(target_train.iloc[:, 1].median(), index=target_test.index)
target_median_final

In [ ]:
target_median = np.column_stack((target_median_rougher, target_median_final))
target_median

Передадим метрике качества массив с медианами.

In [ ]:
sMAPE(target_test, target_median)

Предсказания целевых признаков по медианным значениям приводят к средней ошибке около 9% по взвешенным эффективностям обогащения.

Поскольку модель случайного леса показала значение метрики качества почти на 1 п.п. ниже, то можно сказать, что она прошла проверку на адекватность.

Проверим прошедшую на адекватность лучшую модель на тестовой выборке.

In [ ]:
best_forest_model = RandomForestRegressor(random_state=2, n_estimators=20, max_depth=4, max_features=7)

In [ ]:
best_forest_model.fit(features_train, target_train)

In [ ]:
predictions_forest = best_forest_model.predict(test_sample)

In [ ]:
display(f'sMAPE на тестовой выборке: {sMAPE(target_test, predictions_forest)}')

### Вывод:
Лучшей моделью является случайный лес.

Оценка метрики качества на тестовой выборке - около 8.7%. Значит, если мы будем пользоваться ей в будущем, то в среднем будем ошибаться почти на 9% при оценке эффективности обогащения концентратов.

Отметим, что модель прошла проверку на адекватность - она справляется с предсказанием целевых признаков лучше, чем константная модель с медианами - 8.08% против 9.06% на обучающей выборке. Да, различие невелико, однако оно есть, и это значит, что мы на верном пути.